In [1]:
%load_ext Cython
%matplotlib ipympl

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pykonal
import seispy

HOME         = os.environ['HOME']
GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']

In [2]:
ak135 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/AK135F_AVG.csv'),
    header=None,
    names=('depth', 'density', 'vp', 'vs', 'Q_kappa', 'Q_mu')
)
ak135['radius'] = 6371 - ak135['depth']
ak135 = ak135.sort_values('radius')

In [13]:
src_idx                        = (250, 0, 9)
far_field                      = pykonal.EikonalSolver(coord_sys='spherical')
far_field.vgrid.min_coords     = 3600, np.pi/2, np.pi/4
far_field.vgrid.node_intervals = 10, 1, np.pi/20
far_field.vgrid.npts           = 278, 1, 11
far_field.vv                   = np.ones(far_field.vgrid.npts)

for ir in range(far_field.vgrid.npts[0]):
    far_field.vv[ir] = np.interp(far_field.vgrid[ir,0,0,0], ak135['radius'], ak135['vp'])
    
far_field.uu[src_idx]     = 0
far_field.is_far[src_idx] = False
far_field.close.push(*src_idx)
%time far_field.solve()


pgrid = far_field.pgrid[...]
vgrid = far_field.vgrid[...]
uu = far_field.uu
vv = far_field.vv
if far_field.pgrid.is_periodic[2]:
    pgrid = np.append(pgrid, pgrid[:,:,0].reshape(*pgrid.shape[:2], 1, 3), axis=2)
    vgrid = np.append(vgrid, vgrid[:,:,0].reshape(*vgrid.shape[:2], 1, 3), axis=2)
    uu   = np.append(uu, far_field.uu[...,0].reshape(*far_field.uu.shape[:2], 1), axis=2)
    vv   = np.append(vv, far_field.vv[...,0].reshape(*far_field.vv.shape[:2], 1), axis=2)
xxp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.cos(pgrid[...,2])
yyp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.sin(pgrid[...,2])
xxv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.cos(vgrid[...,2])
yyv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.sin(vgrid[...,2])


plt.close('all')
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1, aspect=1)
qmesh = ax1.pcolormesh(
    xxv[:,0,:],
    yyv[:,0,:],
    vv[:,0,:],
    cmap=plt.get_cmap('jet'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax1, orientation='horizontal')
cbar.set_label('Velocity [km/s]')
ax1.scatter(
    xxv[src_idx],
    yyv[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)
ax2 = fig.add_subplot(1, 2, 2, aspect=1)
qmesh = ax2.pcolormesh(
    xxp[:,0,:],
    yyp[:,0,:],
    uu[:,0,:],
    cmap=plt.get_cmap('hot_r'),
    shading='gouraud',
)
ax2.yaxis.tick_right()
cbar = fig.colorbar(qmesh, ax=ax2, orientation='horizontal')
cbar.set_label('Travel time [s]')
ax2.scatter(
    xxp[src_idx],
    yyp[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)

dd = 10
ax2.quiver(
    xxp[::dd,0,:],
    yyp[::dd,0,:],
    -(
         np.sin(far_field.pgrid[::dd,0,:,1])*np.cos(far_field.pgrid[::dd,0,:,2])*gg[::dd,0,:,0]
        +np.cos(far_field.pgrid[::dd,0,:,1])*np.cos(far_field.pgrid[::dd,0,:,2])*gg[::dd,0,:,1]
        -np.sin(far_field.pgrid[::dd,0,:,2])*gg[::dd,0,:,2]
    ),
    -(
         np.sin(far_field.pgrid[::dd,0,:,1])*np.sin(far_field.pgrid[::dd,0,:,2])*gg[::dd,0,:,0]
        +np.cos(far_field.pgrid[::dd,0,:,1])*np.sin(far_field.pgrid[::dd,0,:,2])*gg[::dd,0,:,1]
        +np.cos(far_field.pgrid[::dd,0,:,2])*gg[::dd,0,:,2]
    ),
    zorder=3
)
# ray = seispy.coords.as_spherical(
#     trace_ray_euler(far_field, (6300, np.pi/2, np.pi/3))
# ).to_cartesian()
# ax2.plot(
#     ray[...,0],
#     ray[...,1]
# )

CPU times: user 16.7 ms, sys: 723 µs, total: 17.4 ms
Wall time: 16.7 ms


FigureCanvasNbAgg()

In [ ]:
(gg[:,0,:,0] * np.sin(gg[:,0,:,1]) )* np.sin(gg[:,0,:,2]))[::10]

In [ ]:
%%cython

# cython:    boundscheck=False
# cython:    cdivision=True
# cython:    language_level=3
# distutils: language=c++

# Python imports
import collections
import itertools
import numpy as np
from pykonal import LinearInterpolator3D

# Cython imports
cimport numpy as np
cimport libc.math
from libcpp.vector cimport vector as cpp_vector
from libc.stdlib   cimport malloc, free

# Define the level of computational precision.
ctypedef np.float64_t _REAL_t
ctypedef np.uint16_t  _UINT_t
DTYPE_REAL = np.float64
DTYPE_UINT = np.uint16

DEF _ERROR_REAL = -999999999999.
ERROR_REAL      = DTYPE_REAL(_ERROR_REAL)

# A simple structure to hold 3D array indices.
cdef struct Index3D:
    Py_ssize_t i1, i2, i3

# A simple Exception class.
class OutOfBoundsError(Exception):
    def __init__(self, msg=''):
        self.msg = msg

def trace_ray_euler(self, start):
    cdef cpp_vector[_REAL_t *]       ray
    cdef _REAL_t                     step_size, gx, gy, gz, norm
    cdef _REAL_t                     *point_new
    cdef _REAL_t[3]                  point_last, point_2last
    cdef Py_ssize_t                  i
    cdef np.ndarray[_REAL_t, ndim=2] ray_np

    point_new = <_REAL_t *> malloc(3 * sizeof(_REAL_t))
    point_new[0], point_new[1], point_new[2] = start
    ray.push_back(point_new)
    # step_size <-- half the smallest node_interval
#     step_size = np.min(
#         [
#             self.pgrid.node_intervals[iax]
#             for iax in range(self.ndim) if iax not in self.iax_null
#         ]
#     ) / 2
    step_size = 10
    # Create an interpolator for the gradient field
    grid       = self.pgrid[...]
    d0, d1, d2 = self.pgrid.node_intervals
    n0, n1, n2 = self.pgrid.npts

    if 0 not in self.iax_null:
        g0 = np.concatenate(
            [
                # Second-order forward difference evaluated along the lower edge
                ((self.uu[2] - 4*self.uu[1] + 3*self.uu[0]) / (2*d0)).reshape(1, n1, n2),
                # Second order central difference evaluated in the interior
                (self.uu[2:] - self.uu[:-2]) / (2*d0),
                # Second order backward difference evaluated along the upper edge
                ((self.uu[-3] - 4*self.uu[-2] + 3*self.uu[-1]) / (2*d0)).reshape(1, n1, n2)
            ],
            axis=0
        )
    else:
        g0 = np.zeros((n0, n1, n2))

    if 1 not in self.iax_null:
        g1 = np.concatenate(
            [
                # Second-order forward difference evaluated along the lower edge
                (
                      (self.uu[:,2] - 4*self.uu[:,1] + 3*self.uu[:,0]) / (2*grid[:,0,:,0]*d1)
                ).reshape(n0, 1, n2),

                # Second order central difference evaluated in the interior
                (self.uu[:,2:] - self.uu[:,:-2]) / (2*grid[:,1:-1,:,0]*d1),
                # Second order backward difference evaluated along the upper edge
                (
                    (self.uu[:,-3] - 4*self.uu[:,-2] + 3*self.uu[:,-1]) / (2*grid[:,-1,:,0]*d1)
                ).reshape(n0, 1, n2)
            ],
            axis=1
        )
    else:
        g1 = np.zeros((n0, n1, n2))

    if 2 not in self.iax_null:
        g2 = np.concatenate(
            [
                # Second-order forward difference evaluated along the lower edge
                (
                      (self.uu[:,:,2] - 4*self.uu[:,:,1] + 3*self.uu[:,:,0]) / (2*grid[:,:,0,0]*np.sin(grid[:,:,0,1])*d2)
                ).reshape(n0, n1, 1),

                # Second order central difference evaluated in the interior
                (self.uu[:,:,2:] - self.uu[:,:,:-2]) / (2*grid[:,:,1:-1,0]*np.sin(grid[:,:,1:-1,0])*d2),
                # Second order backward difference evaluated along the upper edge
                (
                    (self.uu[:,:,-3] - 4*self.uu[:,:,-2] + 3*self.uu[:,:,-1]) / (2*grid[:,:,-1,0]*np.sin(grid[:,:,-1,1])*d2)
                ).reshape(n0, n1, 1)
            ],
            axis=2
        )
    else:
        g2 = np.zeros((n0, n1, n2))
    
    grad_0 = LinearInterpolator3D(self.pgrid, g0.astype(DTYPE_REAL))
    grad_1 = LinearInterpolator3D(self.pgrid, g1.astype(DTYPE_REAL))
    grad_2 = LinearInterpolator3D(self.pgrid, g2.astype(DTYPE_REAL))

    # Create an interpolator for the travel-time field
    uu = LinearInterpolator3D(self.pgrid, self.uu)
    point_last   = ray.back()
    while True:
        g0   = grad_0.interpolate(np.asarray(point_last))
        g1   = grad_1.interpolate(np.asarray(point_last))
        g2   = grad_2.interpolate(np.asarray(point_last))
        norm = libc.math.sqrt(g0**2 + g1**2 + g2**2)
        g0  /= norm
        g1  /= norm
        g2  /= norm
        point_new = <_REAL_t *> malloc(3 * sizeof(_REAL_t))
        point_new[0] = point_last[0] - step_size * g0
        point_new[1] = point_last[1] - step_size * g1
        point_new[2] = point_last[2] - step_size * g2 / (point_last[0] * np.sin(point_last[1]))
        print(point_new[0], point_new[1], point_new[2])
        point_2last  = ray.back()
        ray.push_back(point_new)
        point_last   = ray.back()
        if uu.interpolate(np.asarray(point_2last)) <= uu.interpolate(np.asarray(point_last)):
            break
    ray_np = np.zeros((ray.size()-1, 3), dtype=DTYPE_REAL)
    for i in range(ray.size()-1):
        ray_np[i, 0] = ray[i][0]
        ray_np[i, 1] = ray[i][1]
        ray_np[i, 2] = ray[i][2]
        free(ray[i])
    return (ray_np)

In [5]:
self       = far_field
grid       = self.pgrid[...]
d0, d1, d2 = self.pgrid.node_intervals
n0, n1, n2 = self.pgrid.npts

if 0 not in self.iax_null:
    g0 = np.concatenate(
        [
            # Second-order forward difference evaluated along the lower edge
            ((self.uu[2] - 4*self.uu[1] + 3*self.uu[0]) / (2*d0)).reshape(1, n1, n2),
            # Second order central difference evaluated in the interior
            (self.uu[2:] - self.uu[:-2]) / (2*d0),
            # Second order backward difference evaluated along the upper edge
            ((self.uu[-3] - 4*self.uu[-2] + 3*self.uu[-1]) / (2*d0)).reshape(1, n1, n2)
        ],
        axis=0
    )
else:
    g0 = np.zeros((n0, n1, n2))

if 1 not in self.iax_null:
    g1 = np.concatenate(
        [
            # Second-order forward difference evaluated along the lower edge
            (
                  (self.uu[:,2] - 4*self.uu[:,1] + 3*self.uu[:,0]) / (2*grid[:,0,:,0]*d1)
            ).reshape(n0, 1, n2),

            # Second order central difference evaluated in the interior
            (self.uu[:,2:] - self.uu[:,:-2]) / (2*grid[:,1:-1,:,0]*d1),
            # Second order backward difference evaluated along the upper edge
            (
                (self.uu[:,-3] - 4*self.uu[:,-2] + 3*self.uu[:,-1]) / (2*grid[:,-1,:,0]*d1)
            ).reshape(n0, 1, n2)
        ],
        axis=1
    )
else:
    g1 = np.zeros((n0, n1, n2))

if 2 not in self.iax_null:
    g2 = np.concatenate(
        [
            # Second-order forward difference evaluated along the lower edge
            (
                  (self.uu[:,:,2] - 4*self.uu[:,:,1] + 3*self.uu[:,:,0]) / (2*grid[:,:,0,0]*np.sin(grid[:,:,0,1])*d2)
            ).reshape(n0, n1, 1),

            # Second order central difference evaluated in the interior
            (self.uu[:,:,2:] - self.uu[:,:,:-2]) / (2*grid[:,:,1:-1,0]*np.sin(grid[:,:,1:-1,0])*d2),
            # Second order backward difference evaluated along the upper edge
            (
                (self.uu[:,:,-3] - 4*self.uu[:,:,-2] + 3*self.uu[:,:,-1]) / (2*grid[:,:,-1,0]*np.sin(grid[:,:,-1,1])*d2)
            ).reshape(n0, n1, 1)
        ],
        axis=2
    )
else:
    g2 = np.zeros((n0, n1, n2))

In [7]:
gg = np.moveaxis(np.stack([g0, g1, g2]), 0, -1)

In [ ]:
trace_ray_euler(far_field, (6300, np.pi/2, np.pi/3))

In [ ]:
self = far_field
gg = np.moveaxis(
    np.stack(
        np.gradient(
            self.uu,
            *[
                self.pgrid.node_intervals[iax]
                for iax in range(self.ndim) if iax not in self.iax_null
            ],
            axis=[
                iax
                for iax in range(self.ndim)
                if iax not in self.iax_null
            ]
        )
    ),
    0, -1
)
for iax in self.iax_null:
    gg = np.insert(gg, iax, np.zeros(self.pgrid.npts), axis=-1)

In [ ]:
self = far_field

g0 = (self.uu[1:] - self.uu[:-1]) / self.pgrid.node_intervals[0]
g2 = (self.uu[:,:,1:] - self.uu[:,:,:-1]) / (self.pgrid[:,:,:-1,0] * np.sin(self.pgrid[:,:,:-1,1])* self.pgrid.node_intervals[2])

In [ ]:
# Second-order forward difference evaluated along the lower edge
(self.uu[2] - 4*self.uu[1] + 3*self.uu[0]) / (2*self.pgrid.node_intervals[0])

In [ ]:
# Second order backward difference evaluated along the upper edge
(self.uu[-3] - 4*self.uu[-2] + 3*self.uu[-1]) / (2*self.pgrid.node_intervals[0])

In [ ]:
grid      = self.pgrid[...]
d0, d1, d2 = self.pgrid.node_intervals
n0, n1, n2 = self.pgrid.npts

if 0 not in self.iax_null:
    g0 = np.concatenate(
        [
            # Second-order forward difference evaluated along the lower edge
            ((self.uu[2] - 4*self.uu[1] + 3*self.uu[0]) / (2*d0)).reshape(1, n1, n2),
            # Second order central difference evaluated in the interior
            (self.uu[2:] - self.uu[:-2]) / (2*d0),
            # Second order backward difference evaluated along the upper edge
            ((self.uu[-3] - 4*self.uu[-2] + 3*self.uu[-1]) / (2*d0)).reshape(1, n1, n2)
        ],
        axis=0
    )
else:
    g0 = np.zeros((n0, n1, n2))
    
if 1 not in self.iax_null:
    g1 = np.concatenate(
        [
            # Second-order forward difference evaluated along the lower edge
            (
                  (self.uu[:,2] - 4*self.uu[:,1] + 3*self.uu[:,0]) / (2*grid[:,0,:,0]*d1)
            ).reshape(n0, 1, n2),

            # Second order central difference evaluated in the interior
            (self.uu[:,2:] - self.uu[:,:-2]) / (2*grid[:,1:-1,:,0]*d1),
            # Second order backward difference evaluated along the upper edge
            (
                (self.uu[:,-3] - 4*self.uu[:,-2] + 3*self.uu[:,-1]) / (2*grid[:,-1,:,0]*d1)
            ).reshape(n0, 1, n2)
        ],
        axis=1
    )
else:
    g1 = np.zeros((n0, n1, n2))

if 2 not in self.iax_null:
    g2 = np.concatenate(
        [
            # Second-order forward difference evaluated along the lower edge
            (
                  (self.uu[:,:,2] - 4*self.uu[:,:,1] + 3*self.uu[:,:,0]) / (2*grid[:,:,0,0]*np.sin(grid[:,:,0,1])*d2)
            ).reshape(n0, n1, 1),

            # Second order central difference evaluated in the interior
            (self.uu[:,:,2:] - self.uu[:,:,:-2]) / (2*grid[:,:,1:-1,0]*np.sin(grid[:,:,1:-1,0])*d2),
            # Second order backward difference evaluated along the upper edge
            (
                (self.uu[:,:,-3] - 4*self.uu[:,:,-2] + 3*self.uu[:,:,-1]) / (2*grid[:,:,-1,0]*np.sin(grid[:,:,-1,1])*d2)
            ).reshape(n0, n1, 1)
        ],
        axis=2
    )
else:
    g2 = np.zeros((n0, n1, n2))